# Validação de processos necessários para criação da pipeline

## Instalação dependências

In [5]:
!pip install -q duckdb pandas


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## DuckDB
O DuckDB é eficiente e de alto desempenho. Ao trabalhar com arquivos CSV, por exemplo, ele não precisa carregar o arquivo CSV inteiro na memória antes de poder processá-lo. Em vez disso, o DuckDB pode ler e processar dados do arquivo em tempo real. Para ver isso em ação, vamos usar o conjunto de dados de Atrasos e Cancelamentos de Voos de 2015.

## Leitura de arquivo com o DuckDB

In [20]:
import duckdb
duckdb.sql('SELECT * FROM "data/sample.csv"').df()

,Status,Código da avaliação,Código da Unidade,Unidade,Cidade,Estado,Região/Grupo,Código,Nome do checklist,Autor,...,Resultado,Comentário do item,Imagens,Plataforma,Coordenadas Iniciais,Coordenadas Finais,Comentários Finais,Comentários do responsável da unidade,Planos de Ação,Ações
0,Concluído,152622447,2821694,Generic shop,None,None,None,394104,Generic shop,Leidiane Marques,...,"79,56",None,None,iOS,None,None,Auditoria finalizada na unidade,None,None,None
1,Concluído,152622447,2821694,Generic shop,None,None,None,394104,Generic shop,Leidiane Marques,...,"79,56",Ar-condicionado e teto da cozinha com acúmulo ...,https://app.com.br/evaluations,iOS,None,None,Auditoria finalizada,None,None,None


## Mecanismo de processamento incremental

In [10]:
import duckdb
duckdb.sql('SELECT * FROM "data/sample.csv"').df()

,Status,Código da avaliação,Código da Unidade,Unidade,Cidade,Estado,Região/Grupo,Código,Nome do checklist,Autor,...,Resultado,Comentário do item,Imagens,Plataforma,Coordenadas Iniciais,Coordenadas Finais,Comentários Finais,Comentários do responsável da unidade,Planos de Ação,Ações
0,Concluído,152622447,2821694,Generic shop,None,None,None,394104,Generic shop,Leidiane Marques,...,"79,56",None,None,iOS,None,None,Auditoria finalizada na unidade,None,None,None
1,Concluído,152622447,2821694,Generic shop,None,None,None,394104,Generic shop,Leidiane Marques,...,"79,56",Ar-condicionado e teto da cozinha com acúmulo ...,https://app.com.br/evaluations,iOS,None,None,Auditoria finalizada,None,None,None


In [11]:
import duckdb
duckdb.sql('SELECT * FROM "data/sample2.csv"').df()

,Status,Código da avaliação,Código da Unidade,Unidade,Cidade,Estado,Região/Grupo,Código,Nome do checklist,Autor,...,Resultado,Comentário do item,Imagens,Plataforma,Coordenadas Iniciais,Coordenadas Finais,Comentários Finais,Comentários do responsável da unidade,Planos de Ação,Ações
0,Concluído,152622448,2821695,Generic shop,None,None,None,394104,Generic shop,Ruth Andrade,...,"79,56",Necessário realizar higienização,https://app.com.br/evaluations,iOS,None,None,Auditoria finalizada,None,None,None


In [12]:
duckdb.sql('CREATE TABLE raw AS SELECT * FROM "data/sample.csv"')
duckdb.sql('SELECT * FROM raw')

duckdb.sql('''
SELECT * FROM "data/*.csv"
WHERE "Data de sincronização" > (SELECT MAX("Data de sincronização") FROM raw)
''').df()



,Status,Código da avaliação,Código da Unidade,Unidade,Cidade,Estado,Região/Grupo,Código,Nome do checklist,Autor,...,Resultado,Comentário do item,Imagens,Plataforma,Coordenadas Iniciais,Coordenadas Finais,Comentários Finais,Comentários do responsável da unidade,Planos de Ação,Ações
0,Concluído,152622448,2821695,Generic shop,None,None,None,394104,Generic shop,Ruth Andrade,...,"79,56",Necessário realizar higienização,https://app.com.br/evaluations,iOS,None,None,Auditoria finalizada,None,None,None


## Integração do MinIO e DuckDB

### Criação das credenciais

In [14]:
duckdb.sql("""
CREATE OR REPLACE PERSISTENT SECRET my_secret (
TYPE S3,
REGION 'us-east-1',
KEY_ID 'TESTE123',
SECRET 'TESTE123',
ENDPOINT 'localhost:9000',
USE_SSL 'false',
URL_STYLE 'path');
""")


┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ true    │
└─────────┘

### Escrita para o MinIO

In [16]:
duckdb.sql("""
COPY (SELECT
"Código da avaliação" as id,
Unidade as unidade,
Cidade as cidade,
Autor as autor,
FROM 'data/*.csv') TO 's3://hawkeye/checklist.parquet' (FORMAT 'parquet');
""")

### Leitura arquivo do MinIO

In [18]:
duckdb.sql("""
SELECT
id,
unidade,
cidade,
autor,
FROM 's3://hawkeye/checklist.parquet';
""").df()

,id,unidade,cidade,autor
0,152622447,Generic shop,None,Leidiane Marques
1,152622447,Generic shop,None,Leidiane Marques
2,152622448,Generic shop,None,Ruth Andrade


### Leitura e escrita entre arquivos no MinIO

In [19]:
duckdb.sql("""
COPY (SELECT
id as id2,
unidade as unidade2,
cidade as cidade2,
autor as autor2,
FROM 's3://hawkeye/checklist.parquet') TO 's3://hawkeye/checklist2.parquet' (FORMAT 'parquet');
""")